In [47]:
import numpy as np
import pandas as pd

In [76]:
# Define the KNN class
class KNN:
    def __init__(self, k=3, distance_metric='euclidean', feature_weights = None):
        self.k = k
        self.distance_metric = distance_metric
        self.feature_weights = feature_weights if feature_weights is not None else np.ones(11)  # Default is equal weights

    def fit(self, X_train, y_train):
        # TODO: Implement the fit method
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X):
        # TODO: Implement the predict method
        # Step 1: Get the predicted probabilities
        probas = self.predict_proba(X)

        # Step 2: Convert probabilities to class predictions
        # Class 1 is predicted if the probability of class 1 >= 0.5, else class 0
        predictions = np.argmax(probas, axis=1).astype(int)  # Select the class with the highest probability

        return predictions

    def compute_distance(self, X1, X2):
        # TODO: Implement distance computation based on self.distance_metric
        # Hint: Use numpy operations for efficient computation

        # Print the shapes of X1 and X2 to debug
        #print(f"X1 shape: {X1.shape}, X2 shape: {X2.shape}")

        # Ensure X1 and X2 are numpy arrays
        X1 = np.array(X1)
        X2 = np.array(X2)

        # Ensure both X1 and X2 have at least 2 dimensions
        if X1.ndim == 1:
            X1 = X1.reshape(1, -1)
        if X2.ndim == 1:
            X2 = X2.reshape(1, -1)

        # Debugging: Print the shapes of X1, X2, and feature weights
        #print(f"X1 shape: {X1.shape}, X2 shape: {X2.shape}, Feature weights shape: {self.feature_weights.shape}")

        # Apply feature weights to X1 and X2
        X1_weighted = X1 * self.feature_weights
        X2_weighted = X2 * self.feature_weights

        if self.distance_metric == 'euclidean':
            return np.sqrt(np.sum((X1_weighted[:, np.newaxis] - X2_weighted) ** 2, axis=2))
        elif self.distance_metric == 'manhattan':
            return np.sum(np.abs(X1_weighted[:, np.newaxis] - X2_weighted), axis=2)

    def predict_proba(self, X_test):
        return np.array([self._predict_instance_proba(instance) for instance in X_test])

    def _predict_instance(self, instance):
        # TODO: Implement instance prediction

        # # Calculate distances from this test instance to all training points
        # distances = [self._distance(instance, x_train) for x_train in self.X_train]
        # # Get the indices of the sorted distances
        # nearest_neighbors_indices = np.argsort(distances)[:self.k]
        # # Get the classes of the K nearest neighbors
        # nearest_neighbors = [self.y_train[i] for i in nearest_neighbors_indices]
        # # Majority vote
        # majority_vote = Counter(nearest_neighbors).most_common(1)[0][0]
        # return majority_vote

        # Step 1: Compute distances from the test instance to all training points
        distances = self.compute_distance(np.array([instance]), self.X_train).flatten()
        # Step 2: Find the indices of the k nearest neighbors
        nearest_neighbors_indices = np.argsort(distances)[:self.k]
        # Step 3: Retrieve the labels of the k nearest neighbors
        nearest_neighbors_labels = self.y_train[nearest_neighbors_indices]
        # Step 4: Perform a majority vote manually
        unique_labels, counts = np.unique(nearest_neighbors_labels, return_counts=True)
        # Step 5: Return the label with the highest count (the majority class)
        return unique_labels[np.argmax(counts)]

    def _predict_instance_proba(self, instance):
      # Step 1: Compute distances from the instance to all training points
      distances = self.compute_distance(np.array([instance]), self.X_train).flatten()

      # Step 2: Find the indices of the k nearest neighbors
      nearest_neighbors_indices = np.argsort(distances)[:self.k]

      # Step 3: Get the labels of the k nearest neighbors
      nearest_neighbors_labels = self.y_train[nearest_neighbors_indices]

      # Step 4: Compute weights based on distances (use 1/(distance + epsilon) to avoid division by zero)
      epsilon = 1e-10
      nearest_distances = distances[nearest_neighbors_indices]
      weights = 1 / (nearest_distances + epsilon)

      # Step 5: Calculate the weighted sum for each class
      weighted_votes = np.zeros(2)  # Assuming binary classification (class 0 and class 1)
      for i, label in enumerate(nearest_neighbors_labels):
          label = int(label)
          weighted_votes[label] += weights[i]


      # Step 6: Normalize the weights to get probabilities
      total_weight = np.sum(weights)
      prob_class_0 = weighted_votes[0] / total_weight
      prob_class_1 = weighted_votes[1] / total_weight

      # Step 7: Return the probabilities
      return [prob_class_0, prob_class_1]

In [45]:
# Define data preprocessing function
def preprocess_data(train_path, test_path):
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)

    # TODO: Implement data preprocessing
    # Handle categorical variables, scale features, etc.

    # Ensure column names are consistent
    train_data.columns = train_data.columns.str.strip().str.lower()
    test_data.columns = test_data.columns.str.strip().str.lower()

    # Drop irrelevant columns
    train_data = train_data.drop(columns=['customerid', 'surname', 'id'], errors='ignore')
    test_data = test_data.drop(columns=['customerid', 'surname', 'id'], errors='ignore')

    ## Handle missing values (fill with mean for numerical columns)
    numerical_cols_train = train_data.select_dtypes(include=np.number).columns
    numerical_cols_test = test_data.select_dtypes(include=np.number).columns

    # Ensure that test_data only uses numerical columns present in both datasets
    common_numerical_cols = [col for col in numerical_cols_test if col in numerical_cols_train]

    # # Handle missing values (fill with mean for numerical columns)
    # train_data.fillna(train_data.mean(), inplace=True)
    # test_data.fillna(test_data.mean(), inplace=True)

    # One-Hot Encoding for 'Geography'
    geography_dummies_train = pd.get_dummies(train_data['geography'], drop_first=True)
    geography_dummies_test = pd.get_dummies(test_data['geography'], drop_first=True)

    train_data = pd.concat([train_data.drop(columns=['geography']), geography_dummies_train], axis=1)
    test_data = pd.concat([test_data.drop(columns=['geography']), geography_dummies_test], axis=1)

    # Label Encoding for 'Gender' (Male -> 0, Female -> 1)
    train_data['gender'] = train_data['gender'].map({'male': 0, 'female': 1})
    test_data['gender'] = test_data['gender'].map({'male': 0, 'female': 1})

    # Convert all boolean columns to numeric (int)
    bool_cols_train = train_data.select_dtypes(include='bool').columns
    bool_cols_test = test_data.select_dtypes(include='bool').columns

    train_data[bool_cols_train] = train_data[bool_cols_train].astype(int)
    test_data[bool_cols_test] = test_data[bool_cols_test].astype(int)

    # Manual Min-Max scaling for numerical columns
    #numerical_columns = [col for col in X_train.columns if col not in ['geography', 'gender', 'hascrcard', 'isactivemember']]

    for col in common_numerical_cols:
        col_min = train_data[col].min()
        col_max = train_data[col].max()
        #print("hello, still works here")

        # Apply scaling for both train and test data
        # Check if the column has zero variance to avoid division by zero
        if col_max - col_min == 0:
            # Handle zero variance columns (e.g., set them to 0)
            train_data[col] = 0
            test_data[col] = 0
        else:
            # Apply scaling for both train and test data
            train_data[col] = (train_data[col] - col_min) / (col_max - col_min)
            test_data[col] = (test_data[col] - col_min) / (col_max - col_min)


    # Separate features and labels
    X_train = train_data.drop(columns=['exited']).values  # Features for training
    y_train = train_data['exited'].values  # Labels for training
    X_test = test_data.values  # Features for testing

    # After preprocessing data in preprocess_data
    print(train_data.dtypes)
    print(test_data.dtypes)


    return X_train, y_train, X_test

In [68]:
def accuracy_score(y_true, y_pred):
    correct = sum(y_t == y_p for y_t, y_p in zip(y_true, y_pred))
    return correct / len(y_true)

#calculate Area Under Curve of ROC curve
def calculate_roc_auc(y_true, y_pred_probs):
    # Step 1: Sort by predicted probabilities
    sorted_indices = np.argsort(y_pred_probs)[::-1]  # Sort in descending order
    y_true_sorted = y_true[sorted_indices]
    y_pred_probs_sorted = y_pred_probs[sorted_indices]

    # Step 2: Initialize values
    TPRs = [0]  # True Positive Rates
    FPRs = [0]  # False Positive Rates
    thresholds = [1]  # Start with a threshold of 1 (all classified as negative)

    # Initialize counters
    positives = sum(y_true)  # Total number of actual positives (True Positives + False Negatives)
    negatives = len(y_true) - positives  # Total number of actual negatives (True Negatives + False Positives)

    TP = 0  # True Positives
    FP = 0  # False Positives

    # Step 3: Loop through sorted predictions and calculate TPR, FPR
    for i in range(len(y_true_sorted)):
        if y_true_sorted[i] == 1:
            TP += 1  # Correctly classified as positive (True Positive)
        else:
            FP += 1  # Incorrectly classified as positive (False Positive)

        # Compute the TPR and FPR for the current threshold
        TPR = TP / positives  # True Positive Rate (Recall)
        FPR = FP / negatives  # False Positive Rate

        TPRs.append(TPR)
        FPRs.append(FPR)
        thresholds.append(y_pred_probs_sorted[i])

    # Step 4: Compute the AUC using the trapezoidal rule
    # AUC is the area under the ROC curve
    auc = 0.0
    for i in range(1, len(TPRs)):
        auc += (FPRs[i] - FPRs[i - 1]) * (TPRs[i] + TPRs[i - 1]) / 2  # Trapezoidal area

    return np.array(FPRs), np.array(TPRs), auc

# Example usage:
# y_true = np.array([0, 0, 1, 1])
# y_pred_probs = np.array([0.1, 0.4, 0.35, 0.8])

# FPRs, TPRs, auc = calculate_roc_auc(y_true, y_pred_probs)
# print(f"FPR: {FPRs}")
# print(f"TPR: {TPRs}")
# print(f"AUC: {auc}")


# Define cross-validation function
def cross_validate_accuracy(X, y, knn, n_splits=5):
    # Shuffle the data to ensure random distribution of folds
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    X_shuffled = X[indices]
    y_shuffled = y[indices]

    # Split the data into `n_splits` folds
    fold_size = len(X) // n_splits
    accuracy_scores = []

    for fold in range(n_splits):
        # Define the validation fold
        start = fold * fold_size
        end = start + fold_size if fold != n_splits - 1 else len(X)

        X_val = X_shuffled[start:end]
        y_val = y_shuffled[start:end]

        # Use the remaining data as the training set
        X_train = np.concatenate([X_shuffled[:start], X_shuffled[end:]], axis=0)
        y_train = np.concatenate([y_shuffled[:start], y_shuffled[end:]], axis=0)

        # Train the KNN model
        knn.fit(X_train, y_train)

        # Predict on the validation set
        y_pred = knn.predict(X_val)

        # Calculate accuracy for this fold
        accuracy = accuracy_score(y_val, y_pred)
        accuracy_scores.append(accuracy)

    # Compute the average accuracy score across all folds
    avg_accuracy = np.mean(accuracy_scores)

    return avg_accuracy, accuracy_scores

def cross_validate(X, y, knn, n_splits=5):
    # TODO: Implement cross-validation
    # Compute ROC AUC scores

    # Step 1: Shuffle the data to ensure random distribution of folds
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    X_shuffled = X[indices]
    y_shuffled = y[indices]

    # Step 2: Split the data into `n_splits` folds
    fold_size = len(X) // n_splits
    roc_auc_scores = []

    for fold in range(n_splits):
        # Define the validation fold
        start = fold * fold_size
        end = start + fold_size if fold != n_splits - 1 else len(X)

        X_val = X_shuffled[start:end]
        y_val = y_shuffled[start:end]

        # Use the remaining data as the training set
        X_train = np.concatenate([X_shuffled[:start], X_shuffled[end:]], axis=0)
        y_train = np.concatenate([y_shuffled[:start], y_shuffled[end:]], axis=0)

        # Step 3: Train the KNN model
        knn.fit(X_train, y_train)

        # Check the shape of X_val before making predictions
        #print(f"Fold {fold + 1}: X_val shape = {X_val.shape}")

        # Predict probabilities on the validation set
        y_pred_probs = knn.predict_proba(X_val)[:, 1]  # Predict probability of class 1

        # Compute the ROC AUC score for this fold (only store the AUC value)
        _, _, auc = calculate_roc_auc(y_val, y_pred_probs)
        roc_auc_scores.append(auc)  # Store only the AUC score

    # Compute the average ROC AUC score across all folds
    avg_roc_auc = np.mean(roc_auc_scores)

    return avg_roc_auc, roc_auc_scores

    #     # Step 4: Predict probabilities on the validation set
    #     y_pred_probs = knn.predict_proba(X_val)[:, 1]  # Predict probability of class 1

    #     # Step 5: Compute the ROC AUC score for this fold
    #     roc_auc = calculate_roc_auc(y_val, y_pred_probs)
    #     roc_auc_scores.append(roc_auc)

    # # Step 6: Compute the average ROC AUC score across all folds
    # avg_roc_auc = np.mean(roc_auc_scores)

    # return avg_roc_auc, roc_auc_scores

In [80]:
# Predict results
def save_predictions(knn, X_test, output_file, test_ids):
    predictions = knn.predict(X_test)
    results = pd.DataFrame({'id': test_ids, 'Exited': predictions})
    results.to_csv(output_file, index=False)

#creditscore, geography, gender, age, tenure, balance, numproducts, hascrcard, isactivemember, estimatedsalary
# Define feature weights manually
# Assume 11 features (from your preprocessing step)
feature_weights = np.array([5, 1, 1, 3, 5, 5, 3, 5, 5, 5, 1])  # Adjust these weights as needed
# Load and preprocess data
X, y, X_test = preprocess_data('train.csv', 'test.csv')

# # Create and evaluate model
# knn = KNN(k=5, distance_metric='euclidean')

# # Perform cross-validation
# cv_scores = cross_validate(X, y, knn)
# print("Cross-validation scores:", cv_scores)

# average_roc_auc, roc_auc_scores = cross_validate(X, y, knn)
# print(f"Average ROC AUC across folds: {average_roc_auc}")
# print(f"ROC AUC Scores for each fold: {roc_auc_scores}")

# TODO: hyperparamters tuning

# Initialize variables for hyperparameter tuning
best_k = None
best_distance_metric = None
best_acc = 0

# Define the range of k values and distance metrics to try
k_values = range(10, 30)  # k values to test
distance_metrics = ['euclidean', 'manhattan']

# TODO: Perform cross-validation and hyperparameter tuning
for k in k_values:
    for metric in distance_metrics:
        # Create a KNN model with the current hyperparameters
        knn = KNN(k=k, distance_metric=metric, feature_weights=feature_weights)

        # Perform cross-validation to evaluate the model
        average_acc, acc_scores = cross_validate_accuracy(X, y, knn, n_splits=5)

        #accuracy = accuracy_score(y, knn.predict(X))

        # Print the current hyperparameters and ROC AUC score
        print(f"k: {k}, Distance Metric: {metric}, Average Accuracy: {average_acc}")

        # Check if this is the best ROC AUC score so far
        if average_acc > best_acc:
            best_acc = average_acc
            best_k = k
            best_distance_metric = metric

# Print the best hyperparameters found
print(f"Best k: {best_k}, Best Distance Metric: {best_distance_metric}, Best Accuracy: {best_acc}")

# TODO: Train the model on the full dataset with the best hyperparameters
knn = KNN(k=best_k, distance_metric=best_distance_metric)
knn.fit(X, y)

# Make predictions on the test set
test_predictions = knn.predict(X_test)

test_data = pd.read_csv('test.csv')
test_ids = test_data['id']

# Save test predictions
save_predictions(knn, X_test, 'knn_submission.csv', test_ids)
pd.DataFrame({'id': pd.read_csv('test.csv')['id'], 'Exited': test_predictions}).to_csv('submissions.csv', index=False)

creditscore        float64
gender             float64
age                float64
tenure             float64
balance            float64
numofproducts      float64
hascrcard          float64
isactivemember     float64
estimatedsalary    float64
exited             float64
Germany              int64
Spain                int64
dtype: object
creditscore        float64
gender             float64
age                float64
tenure             float64
balance            float64
numofproducts      float64
hascrcard          float64
isactivemember     float64
estimatedsalary    float64
Germany              int64
Spain                int64
dtype: object
k: 10, Distance Metric: euclidean, Average Accuracy: 0.7978
k: 10, Distance Metric: manhattan, Average Accuracy: 0.7978
k: 11, Distance Metric: euclidean, Average Accuracy: 0.7978000000000001
k: 11, Distance Metric: manhattan, Average Accuracy: 0.7978
k: 12, Distance Metric: euclidean, Average Accuracy: 0.7978
k: 12, Distance Metric: manhattan, Aver